# Table of Contents
 <p><div class="lev2"><a href="#PRECT-statistics"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>PRECT statistics</a></div><div class="lev2"><a href="#CRM_PREC-in-percentile-bins-of-PRECT"><span class="toc-item-num">0.2&nbsp;&nbsp;</span>CRM_PREC in percentile bins of PRECT</a></div>

In [1]:
%load_ext autoreload
%matplotlib inline

In [2]:
%autoreload 2

import pandas as pd
import numpy as np
import numpy.ma as ma
import dask.array as da
import matplotlib.pyplot as plt
from matplotlib import mplDeprecation
import datetime as dt
import sys,os,glob
import matplotlib
from mpl_toolkits.basemap import Basemap
from matplotlib.colors import LogNorm
import pickle

## Add own library to path
workdir = os.getcwd()
thismodule = sys.modules[__name__]
moduledir = os.path.join(os.path.dirname(workdir),'functions')
sys.path.insert(0,moduledir)
print("Own modules available:", [os.path.splitext(os.path.basename(x))[0]
                                 for x in glob.glob(os.path.join(moduledir,'*.py'))])

## Load own libraries
from importingData import *
from scalingApproximations import *
from plotMaps import *
from plot1DInvLog import *
from statisticalDistributions import *
from slicingAndSubsetting import *
from outputResults import *
from CAMsettings import *
from plot2D import *

## Graphical parameters
plt.style.use(os.path.join(matplotlib.get_configdir(),'stylelib/presentation.mplstyle'))

Own modules available: ['CAMsettings', 'daskOptions', 'environmentAndDirectories', 'importingData', 'outputResults', 'physicalConstants', 'plot1DInvLog', 'plot2D', 'plotMaps', 'scalingApproximations', 'slicingAndSubsetting', 'statisticalDistributions', 'thermoConstants', 'thermoFunctions']


In [10]:
time_stride = '1h'
resolution = '1dx'

ranks_label = 'Q8-Q41'
dates_label_full = '185005010100-185105010000'
dates_label_med = '185005010100-185009010000'
dates_label_short = '185005010100-185005020000'
dates_label = dates_label_short
dates = tuple(dates_label.split('-'))

In [4]:
print("choose run options")
scaling = 'omega500tsps'
compset = 'FSPCAMm_AMIP'
experiment = 'piControl'
member = 'r1i1p1'
subset = 'tropics'
# time_ref = '1h'
# res_ref = '1dx'
daskarray = False

choose run options


In [6]:
# Figure output directory
figdir = os.path.join(os.path.dirname(workdir),'figures','omega500tsps',compset,experiment,member,subset)
handle= 'h0'

print()
print("get inputpaths")

# Local input directory
historyFilesSettings = getCAMHistoryFilesSettings()
inputdir, inputdir_processed_day, inputdir_processed_1hr, inputdir_results, inputdir_fx = \
    getInputDirectories(compset,experiment)


get inputpaths


In [13]:
print("load physical variables data")

pr_id = 'PRECT'
crmpr_id = 'CRM_PREC'

pr,crmpr = getValues([pr_id,crmpr_id],
              compset,subset,experiment,time_stride,resolution,
              daskarray=daskarray,dates=dates,handle=handle)

load physical variables data
Importing PRECT, CRM_PREC from 24 history files between 1850-05-01-03600 and 1850-05-02-00000


## PRECT statistics

In [14]:
%%time
print("Compute ranks and ranks locations")

n_pts = pr.size
Q_IL = getInvLogRanks(n_pts,n_pts_per_bin=1,fill_last_decade=True)
i_Q = indexOfRank(99.9,Q_IL)
iQ_slice = slice(i_Q-5,i_Q+5)
ranks, centers, bins = computePercentilesAndBinsFromRanks(pr.flatten(),Q_IL)
iQ_min = 8
iQ_max = min(len(Q_IL),41)
# iQ_max = min(len(Q_IL),iQ_slice.stop)

# Define reference percentiles and compute percentiles
targetranks = Q_IL[iQ_min:iQ_max]
ranks_ref = Q_IL[:iQ_max]
percentiles = adjustRanks(centers,ranks,ranks_ref)

# Suffix to save figures and results
output_suffix = 'Q%d-Q%d_%s'%(iQ_min,iQ_max,'-'.join(dates))

rank_locations= {}
for rank in ranks_ref:
    rank_id  = "%2.4f"%rank
    print(rank_id,end=' ')
    rank_locations[rank_id] = getRankLocations(rank,pr,ranks,bins,rank_locations)
print()

Compute ranks and ranks locations
0.0000 20.5672 36.9043 49.8813 60.1893 68.3772 74.8811 80.0474 84.1511 87.4107 90.0000 92.0567 93.6904 94.9881 96.0189 96.8377 97.4881 98.0047 98.4151 98.7411 99.0000 99.2057 99.3690 99.4988 99.6019 99.6838 99.7488 99.8005 99.8415 99.8741 99.9000 99.9206 99.9369 99.9499 99.9602 99.9684 99.9749 99.9800 99.9842 99.9874 99.9900 
CPU times: user 26.5 ms, sys: 14.6 ms, total: 41.1 ms
Wall time: 42.9 ms


In [15]:
print("compute sample size in each bin")
N_prQ = sampleSizeAtAllRanks(targetranks,pr,ranks_ref,rank_locations=rank_locations)
print(np.nansum(N_prQ),pr.size)

compute sample size in each bin
24413.0 110592


## CRM_PREC in percentile bins of PRECT

In [25]:
x = np.linspace(1,6,6)
print(x[:,np.newaxis,np.newaxis])

[[[ 1.]]

 [[ 2.]]

 [[ 3.]]

 [[ 4.]]

 [[ 5.]]

 [[ 6.]]]


In [28]:
stencil_Q = getRankLocations(99.9,pr,ranks,bins,rank_locations)
print(stencil_Q.shape, crmpr.shape)

(24, 32, 144) (24, 1, 32, 32, 144)
